In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [2]:
df = pd.read_csv('ab_test_results_aggregated_views_clicks.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  80000 non-null  int64  
 1   group    80000 non-null  object 
 2   views    80000 non-null  float64
 3   clicks   80000 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
df.head()

,user_id,group,views,clicks
0,1,control,3.0,0.0
1,2,control,1.0,0.0
2,3,control,3.0,1.0
3,4,control,5.0,0.0
4,5,control,2.0,0.0


In [5]:
df.group.unique()

array(['control', 'test'], dtype=object)

## Sample Ratio Mismatch Checking (SRM)

In [22]:
df.groupby('group')['user_id'].nunique()

group
control    40000
test       40000
Name: user_id, dtype: int64

In [98]:
def sample_ratio_mismatch(control_sample_size, treatment_sample_size, 
                          control_expected_proportion, treatment_expected_proportion):
    
    total_sample_size = control_sample_size + treatment_sample_size
    
    # Compute the expected frequencies under the assumption of equal proportions
    expected_control_successes = control_expected_proportion * total_sample_size
    expected_treatment_successes = treatment_expected_proportion * total_sample_size
    
    # Create the observed and expected frequency arrays
    observed_frequencies = [control_sample_size, treatment_sample_size]
    expected_frequencies = [expected_control_successes, expected_treatment_successes]
    
    # Perform chi-square test to check for sample ratio mismatch
    _, p_value = stats.chisquare(observed_frequencies, expected_frequencies)
    
    # Check if the p-value is less than the significance level (e.g., 0.05)
    if p_value < 0.05:
        return True  # Sample ratio mismatch detected
    else:
        return False  # Sample ratio mismatch not detected

In [99]:
#Usage
control_expected_proportion = 0.5 
treatment_expected_proportion = 1-control_expected_proportion
control_sample_size = df.groupby('group')['user_id'].nunique()['control']
treatment_sample_size = df.groupby('group')['user_id'].nunique()['test']

if sample_ratio_mismatch(control_sample_size, treatment_sample_size, control_expected_proportion, treatment_expected_proportion):
    print("Sample ratio mismatch detected!")
else:
    print("Sample ratio mismatch not detected.")


Sample ratio mismatch not detected.


## Descriptive Analysis

In [103]:
# 1. Total users per variant_name
total_users = df.groupby('group')['user_id'].nunique()

# 2. Average clicks and views per variant_name
average_clicks = df.groupby('group')['clicks'].mean()
average_views = df.groupby('group')['views'].mean()

# 3. Total clicks / total views per variant_name
clicks_to_views_ratio = df.groupby('group').apply(lambda x: x['clicks'].sum() / x['views'].sum())

# 4. Median clicks and views per variant_name
median_clicks = df.groupby('group')['clicks'].median()
median_views = df.groupby('group')['views'].median()

# 4. Median clicks and views per variant_name
q3_clicks = df.groupby('group')['clicks'].quantile(0.75)
q3_views = df.groupby('group')['views'].quantile(0.75)

# Creating a new DataFrame to store the aggregated results
aggregated_data = pd.DataFrame({
    'Total Users': total_users,
    'Average Clicks': average_clicks,
    'Average Views': average_views,
    'Clicks to Views Ratio': clicks_to_views_ratio,
    'Median Clicks': median_clicks,
    'Median Views': median_views,
    'Quartile 3 Clicks':q3_clicks,
    'Quartile 3 Views':q3_views,
})

# Displaying the aggregated data
aggregated_data


,Total Users,Average Clicks,Average Views,Clicks to Views Ratio,Median Clicks,Median Views,Quartile 3 Clicks,Quartile 3 Views
group,,,,,,,,
control,40000,0.345725,4.981050,0.069408,0.0,3.0,1.0,6.0
test,40000,0.401150,5.015475,0.079982,0.0,3.0,1.0,6.0


## Inferential Analysis

**H<sub>0</sub> (null hypothesis)**: There is no difference between control and treatment variant

**H<sub>1</sub> (alternative hypothesis)**: There is a difference between control and treatment variant

**Hypothesis Testing Method**: z-test proportion

In [94]:
from statsmodels.stats.proportion import proportions_ztest

def ztest_calculation(control_view, control_click, treatment_view, treatment_click):
    count = np.array([control_click, treatment_click]) #numerator
    nobs = np.array([control_view, treatment_view]) #denom
    stat, pval = proportions_ztest(count, nobs)
    
    control_metric = control_click/control_view
    treatment_metric = treatment_click/treatment_view
    
    if pval <0.05: 
        significant_test = 'There is difference between control and treatment'
    else:
        significant_test = 'There is no difference between control and treatment'
    
    result = {'control':control_metric*100,
             'treatment':treatment_metric*100,
             '%difference':(treatment_metric - control_metric)*100/control_metric,
             'p-value':'{0:0.5f}'.format(pval),
             'conclusion':significant_test}

    return pd.DataFrame(result, index=[0])

In [95]:
control_view = df.loc[df['group']=='control','views'].sum()
control_click = df.loc[df['group']=='control','clicks'].sum()
treatment_view = df.loc[df['group']=='test','views'].sum()
treatment_click = df.loc[df['group']=='test','clicks'].sum()

result = ztest_calculation(control_view, control_click, treatment_view, treatment_click)

# print('{0:0.3f}'.format(pval))
result

,control,treatment,%difference,p-value,conclusion
0,6.940806,7.998245,15.235116,0.00000,There is difference between control and treatment
